# Integration
After ther first touch of looking to the data in 

https://github.com/JHC90/HandsOnMachineLearningSciKitTensorFlow/blob/master/Own-Examples/1_DataExploring.ipynb


we split the data into train test and 


In [1]:
import DownloadAndExtractAndLoadDatasets as DEL # my own Module
from platform import python_version
import pandas as pd
import numpy as np
import hashlib
from sklearn.model_selection import StratifiedShuffleSplit

print(python_version())

import types
def imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            yield val.__name__,
list(imports())

3.7.4


[('builtins',),
 ('builtins',),
 ('DownloadAndExtractAndLoadDatasets',),
 ('pandas',),
 ('numpy',),
 ('hashlib',),
 ('types',)]

In [2]:

DEL.fetch_housing_data()

In [3]:
WorkingData = DEL.load_housing_data()

In [4]:
print(type(WorkingData))

<class 'pandas.core.frame.DataFrame'>


# Splitting Process

In [5]:
def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [6]:
train_set, test_set = split_train_test(WorkingData, 0.2) # hier die aufteilung // hier 20 % testdata
print(len(train_set), "train +", len(test_set), "test")

16512 train + 4128 test


In [7]:
train_set.head(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
331,-122.17,37.74,41.0,1613.0,445.0,1481.0,414.0,2.4028,97700.0,NEAR BAY
1766,-122.34,37.95,44.0,1675.0,317.0,806.0,311.0,3.0694,135300.0,NEAR BAY
3797,-118.37,34.16,17.0,4150.0,1148.0,1808.0,1041.0,3.5051,232400.0,<1H OCEAN
6992,-118.02,33.95,38.0,2139.0,426.0,1138.0,412.0,4.2917,168900.0,<1H OCEAN
2189,-120.08,36.72,22.0,1339.0,251.0,820.0,276.0,3.6000,83200.0,INLAND


# seeds
Above solution will always bring up a new set of train and testdata // nevertheless there will be the same amount of Instances but always shuffled. For building Models we want to use the same sets => therefore we will use the Seed to get the same sets every time

This seed is good for Offline/Batch Learning with no changes in the dataset

So we rewrite the funkction and add folwoing line....
in the next step we will call the function several times and will see always the same output


In [8]:
def split_train_test(data, test_ratio):
    np.random.seed(42)
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [9]:
train_set, test_set = split_train_test(WorkingData, 0.2) # hier die aufteilung // hier 20 % testdata
train_set.head(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
14196,-117.03,32.71,33.0,3126.0,627.0,2300.0,623.0,3.2596,103000.0,NEAR OCEAN
8267,-118.16,33.77,49.0,3382.0,787.0,1314.0,756.0,3.8125,382100.0,NEAR OCEAN
17445,-120.48,34.66,4.0,1897.0,331.0,915.0,336.0,4.1563,172600.0,NEAR OCEAN
14265,-117.11,32.69,36.0,1421.0,367.0,1418.0,355.0,1.9425,93400.0,NEAR OCEAN
2271,-119.80,36.78,43.0,2382.0,431.0,874.0,380.0,3.5542,96500.0,INLAND


In [10]:
train_set, test_set = split_train_test(WorkingData, 0.2) # hier die aufteilung // hier 20 % testdata
train_set.head(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
14196,-117.03,32.71,33.0,3126.0,627.0,2300.0,623.0,3.2596,103000.0,NEAR OCEAN
8267,-118.16,33.77,49.0,3382.0,787.0,1314.0,756.0,3.8125,382100.0,NEAR OCEAN
17445,-120.48,34.66,4.0,1897.0,331.0,915.0,336.0,4.1563,172600.0,NEAR OCEAN
14265,-117.11,32.69,36.0,1421.0,367.0,1418.0,355.0,1.9425,93400.0,NEAR OCEAN
2271,-119.80,36.78,43.0,2382.0,431.0,874.0,380.0,3.5542,96500.0,INLAND


# Problem with online learning
Each time we update the Dataset we are learnign from we have a Problem in the splitting. We want the delta of new data be Shared to training and testin adaequat. Following we want to create on testset and each time we get a new Dataset we will choose the same Instances including 20% of the new Data. 


Roadmap
* Add Index to the data (if-Necessay) // test_set_check
* based on the index build the Split // split_train_test_by_id

In [11]:
def test_set_check(identifier, test_ratio, hash):
    return hash(np.int64(identifier)).digest()[-1] < 256 * test_ratio
def split_train_test_by_id(data, test_ratio, id_column, hash=hashlib.md5):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio, hash))
    return data.loc[~in_test_set], data.loc[in_test_set]

In [12]:
housing_with_id = WorkingData.reset_index() # adds an `index` column
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "index")

# Problem with the Proportion in the population
So far we just checked the random sampling, now we will check the stratiefied sampling method

In [13]:
WorkingData["income_cat"] = np.ceil(WorkingData["median_income"] / 1.5)
WorkingData["income_cat"].where(WorkingData["income_cat"] < 5, 5.0, inplace=True)

In [14]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(WorkingData, WorkingData["income_cat"]):
    strat_train_set = WorkingData.loc[train_index]
    strat_test_set = WorkingData.loc[test_index]